In [1]:
import os, random
import numpy as np
import pandas as pd
import scipy.io as sio
from ieeg.auth import Session

from get_iEEG_data import *
from spike_detector import *
from iEEG_helper_functions import *

ERIN_DIRECTORY = "../../../../erinconr/projects/fc_toolbox/results/all_out"

## Patient selection

In [2]:
good_hup_ids_for_spike_detector = np.load("../good_hup_ids_for_spike_detector.npy")
good_hup_ids_for_spike_detector

array([137, 138, 139, 140, 141, 142, 143, 145, 146, 148, 150, 151, 152,
       153, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
       167, 168, 169, 170, 171, 172, 173, 174, 175, 177, 178, 179, 180,
       181, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 196, 197,
       199, 201, 202, 204, 205, 206, 207, 209, 210, 211, 213, 214, 215,
       219, 221, 223, 224, 225])

In [3]:
# Load HUP_implant_dates.xlsx
nina_patients_df = pd.read_excel("../../../Data/HUP_implant_dates.xlsx")
# Make the hup_id column integers
nina_patients_df["hup_id"] = nina_patients_df["hup_id"].astype(int)
nina_patients_df

,hup_id,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,225,HUP225_phaseII,2021-10-18,07:15:00,2021-10-26 17:30:00,58.5
1,224,HUP224_phaseII,2021-10-13,07:15:00,2021-10-20 00:00:00,85.5
2,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
3,221,HUP221_phaseII,2021-08-16,07:15:00,2021-08-23 00:00:00,124.3
4,219,HUP219_phaseII,2021-07-12,07:15:00,2021-07-16 08:18:00,101.6
...,...,...,...,...,...,...
75,141,HUP141_phaseII,2017-05-24,07:15:00,2017-06-01 00:00:00,85.7
76,140,HUP140_phaseII_D01-D02,2017-05-10,07:15:00,2017-05-19 00:00:00,56.7
77,139,HUP139_phaseII,2017-04-26,07:15:00,2017-05-09 00:00:00,69.8
78,138,HUP138_phaseII,2017-04-12,07:15:00,2017-04-20 00:00:00,84.4


In [4]:
# Add a boolean column in nina_patients_df called is_single_dataset and make it True if IEEG_Portal_Number ends with "phaseII"
nina_patients_df["is_single_dataset"] = nina_patients_df[
    "IEEG_Portal_Number"
].str.endswith("phaseII")
# Add a boolean column in nina_patients_df called is_good_for_spike_detector and make it True if the row's hup_id is in good_hup_ids_for_spike_detector
nina_patients_df["is_good_for_spike_detector"] = nina_patients_df["hup_id"].isin(
    good_hup_ids_for_spike_detector
)

In [5]:
# Drop the rows in nina_patients_df where is_single_dataset is False
nina_patients_df = nina_patients_df[nina_patients_df.is_single_dataset == True]
# Drop the rows in nina_patients_df where is_good_for_spike_detector is False
nina_patients_df = nina_patients_df[nina_patients_df.is_good_for_spike_detector == True]
# Sort by hup_id in ascending order
nina_patients_df = nina_patients_df.sort_values(by=["hup_id"], ascending=True)
# Drop columns Implant_Date, implant_time, Explant_Date, weight_kg
nina_patients_df = nina_patients_df.drop(
    columns=["Implant_Date", "implant_time", "Explant_Date", "weight_kg"]
)
# Reset index
nina_patients_df = nina_patients_df.reset_index(drop=True)
nina_patients_df

,hup_id,IEEG_Portal_Number,is_single_dataset,is_good_for_spike_detector
0,138,HUP138_phaseII,True,True
1,139,HUP139_phaseII,True,True
2,141,HUP141_phaseII,True,True
3,142,HUP142_phaseII,True,True
4,143,HUP143_phaseII,True,True
5,145,HUP145_phaseII,True,True
6,146,HUP146_phaseII,True,True
7,150,HUP150_phaseII,True,True
8,151,HUP151_phaseII,True,True
9,154,HUP154_phaseII,True,True


## Main loop

In [6]:
print("Using Carlos session")
with open("agu_ieeglogin.bin", "r") as f:
    session = Session("aguilac", f.read())

Using Carlos session


In [7]:
for index, row in nina_patients_df.iterrows():
    error_percetnages = []
    num_of_clips_overcounting = 0
    num_of_clips_undercounting = 0

    hup_id = row["hup_id"]
    dataset_name = row["IEEG_Portal_Number"]

    print("\n")
    print(f"------Processing HUP {hup_id} with dataset {dataset_name}------")

    ########################################
    # Get the data from IEEG
    ########################################

    dataset = session.open_dataset(dataset_name)

    all_channel_labels = np.array(dataset.get_channel_labels())
    channel_labels_to_download = all_channel_labels[
        electrode_selection(all_channel_labels)
    ]

    duration_usec = dataset.get_time_series_details(
        channel_labels_to_download[0]
    ).duration
    duration_hours = int(duration_usec / 1000000 / 60 / 60)
    enlarged_duration_hours = duration_hours + 24

    print(f"Opening {dataset_name} with duration {duration_hours} hours")

    ########################################
    # Process Erin's file
    ########################################

    erin_mat_file = sio.loadmat(os.path.join(ERIN_DIRECTORY, f"HUP{hup_id}_pc.mat"))

    mat_content = erin_mat_file["pc"]
    mat_content.dtype

    name = mat_content[0, 0]["name"]
    file = mat_content[0, 0]["file"]
    file.dtype

    name = file[0, 0]["name"]
    run = file[0, 0]["run"]
    run.dtype

    data = run[0]["data"]
    run_times = run[0]["run_times"]
    block_times = run[0]["block_times"]
    cohere_out = run[0]["cohere_out"]

    assert len(data) == len(run_times) == len(block_times) == len(cohere_out) > 100
    # Taking first 2 and last 2 elements
    first_two = run_times[:2].tolist()
    last_two = run_times[-2:].tolist()

    # Selecting 16 random elements from the middle and sorting them
    middle_indices = sorted(random.sample(range(2, len(run_times) - 2), 18))
    middle_elements = [run_times[i] for i in middle_indices]

    # Combining the lists
    all_indices = [0, 1] + middle_indices
    all_elements = first_two + middle_elements
    assert len(all_indices) == len(all_elements) == 20

    # Iterating through the selected elements and their indices
    for i, clip in zip(all_indices, all_elements):
        clip = clip[0]
        start_time_sec, end_time_sec = clip
        start_time_sec = int(start_time_sec)
        end_time_sec = int(end_time_sec)
        start_time_usec = start_time_sec * 1000000
        end_time_usec = end_time_sec * 1000000
        montage = data[i][0]["montage"][0]["name"][0][1][0]
        assert montage == "car"
        spikes = data[i][0]["montage"][0]["spikes"][0][1]
        num_spikes = len(spikes)
        print(f"Clip {i}, {start_time_sec} to {end_time_sec}")

        try:
            ieeg_data, fs = get_iEEG_data(
                "aguilac",
                "agu_ieeglogin.bin",
                dataset_name,
                start_time_usec,
                end_time_usec,
                channel_labels_to_download,
            )
            fs = int(fs)
        except:
            continue

        # Check if ieeg_data dataframe is all NaNs
        if ieeg_data.isnull().values.all():
            print(
                f"Empty dataframe after download, skip... There should be {num_spikes} spikes"
            )
            continue

        good_channels_res = detect_bad_channels_optimized(ieeg_data.to_numpy(), fs)
        good_channel_indicies = good_channels_res[0]
        good_channel_labels = channel_labels_to_download[good_channel_indicies]
        ieeg_data = ieeg_data[good_channel_labels].to_numpy()

        # Check if ieeg_data is empty after dropping bad channels
        if ieeg_data.size == 0:
            print(
                f"Empty dataframe after artifact rejection, skip... There should be {num_spikes} spikes"
            )
            continue

        ieeg_data = common_average_montage(ieeg_data)

        # Apply the filters directly on the DataFrame
        ieeg_data = notch_filter(ieeg_data, 59, 61, fs)
        ieeg_data = bandpass_filter(ieeg_data, 1, 70, fs)

        ##############################
        # Detect spikes
        ##############################

        spike_output = spike_detector(
            data=ieeg_data,
            fs=fs,
            electrode_labels=good_channel_labels,
        )
        if len(spike_output) == 0:
            print(
                f"No spikes detected, skip saving... There should be {num_spikes} spikes"
            )
            continue
        else:
            print(f"Detected {len(spike_output)} spikes, should be {num_spikes} spikes")
            if len(spike_output) > num_spikes:
                num_of_clips_overcounting += 1
            elif len(spike_output) < num_spikes:
                num_of_clips_undercounting += 1

            error_percentage = abs(len(spike_output) - num_spikes) / num_spikes
            error_percetnages.append(error_percentage)

    print(
        f"Patient HUP {hup_id} average error percentages: {np.mean(error_percetnages)*100}%"
    )
    print(f"Patient HUP {hup_id} overcounting clips: {num_of_clips_overcounting}")
    print(f"Patient HUP {hup_id} undercounting clips: {num_of_clips_undercounting}")



------Processing HUP 138 with dataset HUP138_phaseII------
Opening HUP138_phaseII with duration 172 hours
Clip 0, 87 to 147
Empty dataframe after download, skip... There should be 0 spikes
Clip 1, 1096 to 1156
Empty dataframe after artifact rejection, skip... There should be 0 spikes
Clip 30, 18094 to 18154
Detected 6 spikes, should be 6 spikes
Clip 33, 19954 to 20014
No spikes detected, skip saving... There should be 0 spikes
Clip 97, 58376 to 58436
Detected 10 spikes, should be 10 spikes
Clip 156, 93606 to 93666
Detected 2 spikes, should be 2 spikes
Clip 190, 114364 to 114424
No spikes detected, skip saving... There should be 0 spikes
Clip 193, 116145 to 116205
No spikes detected, skip saving... There should be 0 spikes
Clip 227, 136323 to 136383
No spikes detected, skip saving... There should be 0 spikes
Clip 256, 154027 to 154087
Detected 2 spikes, should be 4 spikes
Clip 278, 167188 to 167248
Detected 27 spikes, should be 27 spikes
Clip 376, 225847 to 225907
No spikes detected, 

/mnt/leif/littlab/users/devinma/Libraries/miniconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/leif/littlab/users/devinma/Libraries/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Patient HUP 211 average error percentages: nan%
Patient HUP 211 overcounting clips: 0
Patient HUP 211 undercounting clips: 0


------Processing HUP 219 with dataset HUP219_phaseII------
Opening HUP219_phaseII with duration 61 hours
Clip 0, 440 to 500
Clip 1, 1090 to 1150
No spikes detected, skip saving... There should be 0 spikes
Clip 15, 9076 to 9136
No spikes detected, skip saving... There should be 0 spikes
Clip 19, 11919 to 11979
Detected 31 spikes, should be 31 spikes
Clip 47, 28549 to 28609
Detected 11 spikes, should be 9 spikes
Clip 63, 38178 to 38238
Detected 11 spikes, should be 11 spikes
Clip 102, 61368 to 61428
Detected 32 spikes, should be 33 spikes
Clip 130, 78233 to 78293
Detected 20 spikes, should be 21 spikes
Clip 133, 79942 to 80002
Detected 28 spikes, should be 28 spikes
Clip 167, 100717 to 100777
Detected 10 spikes, should be 10 spikes
Clip 172, 103644 to 103704
Detected 8 spikes, should be 8 spikes
Clip 184, 110771 to 110831
Detected 29 spikes, should be 29 spikes
C

/mnt/leif/littlab/users/devinma/Libraries/miniconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/mnt/leif/littlab/users/devinma/Libraries/miniconda3/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Patient HUP 224 average error percentages: nan%
Patient HUP 224 overcounting clips: 0
Patient HUP 224 undercounting clips: 0


------Processing HUP 225 with dataset HUP225_phaseII------
Opening HUP225_phaseII with duration 168 hours
Clip 0, 267 to 327
Detected 7 spikes, should be 7 spikes
Clip 1, 1066 to 1126
Empty dataframe after artifact rejection, skip... There should be 0 spikes
Clip 52, 31694 to 31754
Detected 6 spikes, should be 6 spikes
Clip 73, 44170 to 44230
Detected 2 spikes, should be 2 spikes
Clip 132, 79683 to 79743
Detected 2 spikes, should be 2 spikes
Clip 141, 84857 to 84917
No spikes detected, skip saving... There should be 2 spikes
Clip 197, 118402 to 118462
Detected 23 spikes, should be 23 spikes
Clip 227, 136607 to 136667
Detected 2 spikes, should be 2 spikes
Clip 269, 161783 to 161843
No spikes detected, skip saving... There should be 0 spikes
Clip 304, 182496 to 182556
Detected 11 spikes, should be 11 spikes
Clip 322, 193445 to 193505
Detected 17 spikes, should be

Verify the following 20 time clips:
1. The first 2 clips
2. Random 18 clips in the middle (until the last two)

The reason is that the last two might be incomplete.

1. First, make sure the number of spikes are the same or smaller
2. Second, make sure the channels match up
3. Fuzzy time